# WRI Global Power Plant Database Ingestion Pipeline: Step 1 - Data Extraction
Copyright (C) 2022 OS-Climate

This notebook covers the first step of the pipeline:
* Load external source CSV source
* Connect to pachyderm service, create a new repository if need be and check in source data
* Verify that the file can be accessed for the next step which is data loading

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

# Python client for Pachyderm
%pip install python_pachyderm

In [2]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [3]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import osc_ingest_trino as osc
import python_pachyderm

Define Environment and Execution Variables

In [4]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Read the data directory holding raw source data, and list all files

In [6]:
path = '../data/raw/global_power_plant_database_v_1_3/'
files = os.listdir(path)
for f in files:
    print(f)

A_Global_Database_of_Power_Plants.pdf
global_power_plant_database.csv
.ipynb_checkpoints
RELEASE_NOTES.txt
Estimating_Power_Plant_Generation_in_the_Global_Power_Plant_Database.pdf
README.txt


Connect to Pachyderm service using the Python client, create a new repo and check in the source files

In [7]:
# Create a pachyderm client by manually setting up host and port
client = python_pachyderm.Client(os.environ['PACH_ENDPOINT'], os.environ['PACH_PORT'])

In [8]:
# Create a new pachyderm repo called `gppd`
client.create_repo("wri-gppd")

In [9]:
# Display list of repos
print(list(client.list_repo()))

[repo {
  name: "wri-gppd"
  type: "user"
}
created {
  seconds: 1659790074
  nanos: 399079000
}
]


In [10]:
with client.commit("wri-gppd", "master") as commit:
        # Add all the files, recursively inserting from the directory
        # Alternatively, we could use `client.put_file_url` or
        # `client_put_file_bytes`.
        python_pachyderm.put_files(client, path, commit, "/global_power_plant_database_v_1_3/")

In [11]:
# Check the file commit
print(list(client.list_file(("wri-gppd","master"), "/global_power_plant_database_v_1_3/")))

[file {
  commit {
    branch {
      repo {
        name: "wri-gppd"
        type: "user"
      }
      name: "master"
    }
    id: "5158a5ececd8444ba3f578d277720895"
  }
  path: "/global_power_plant_database_v_1_3/.ipynb_checkpoints/"
}
file_type: DIR
committed {
  seconds: 1659790092
  nanos: 6078000
}
size_bytes: 11973385
hash: "-B\204\010\340\215[\206GA\300h 6\360\202\360\226.\301fR\242/W\203\377\034\365\257\375\304"
, file {
  commit {
    branch {
      repo {
        name: "wri-gppd"
        type: "user"
      }
      name: "master"
    }
    id: "5158a5ececd8444ba3f578d277720895"
  }
  path: "/global_power_plant_database_v_1_3/A_Global_Database_of_Power_Plants.pdf"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1659790092
  nanos: 6078000
}
size_bytes: 1441800
hash: ".M\370\311\026`\334\001\000?\342>B\0351\373\021\322\276\205\025\231\200\262\270\"H\330|\260\314\037"
, file {
  commit {
    branch {
      repo {
        name: "wri-gppd"
        type: "user"
      

In [12]:
# Delete pachyderm repo called `wri-gppd`
# Only required if you need to run the pipeline again
# client.delete_repo("wri-gppd")